### Задача 1
Попробуем себя в решении задачи определения темы текста. Будем считать, что два текста похожи по теме, если у них больше общих слов (только не предлогов с союзами), чем у других текстов. У нашей программы для определения темы будет несколько готовых текстов (достаточно больших!) с уже известной темой в базе: выберите тексты (и темы) самостоятельно, 5-6 будет достаточно.

Что должна делать программа? При запуске вы ей сообщаете название нового файла с текстом, который нужно классифицировать, она его открывает, обрабатывает и сравнивает с текстами в своей базе. С которым из текстов оказалось больше всего общих слов, того и тема! Очевидно, вам понадобится какие-то слова из текстов отбрасывать (подумайте, каким образом это сделать - здесь на самом деле несколько вариантов концепций), а еще лемматизировать или хотя бы использовать стемминг.

In [50]:
import os
import spacy


# чтение файла
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# обработка текста
def preprocess_text(text, nlp):
    doc = nlp(text) # токенизация
    lemmatized_tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in nlp.Defaults.stop_words] # заносим в список буквенные токены кроме стоп-слов
    return set(lemmatized_tokens) # возвращает мн-во лемматизированных токенов

# сравнение текстов
def count_similarity(database_texts, new_text, nlp): # принимает database_texts (словарь из темы-текста), текст из нового файла, nlp
    new_text_set = preprocess_text(new_text, nlp) # предобработка нового текста; вернется мн-во лемматизированных токенов

    similarity = [] # сюда складываем кортежи, состящие из темы и степени близости (кол-ва пересекающихся слов) 
    for topic, database_text in database_texts.items(): # итерация по словарю database_texts
        database_text_set = preprocess_text(database_text, nlp) # предобработка текста из базы
        common_words = len(new_text_set.intersection(database_text_set)) # длина мн-ва пересекающихся слов (т.е. кол-во пересекающихся слов)
        similarity.append((topic, common_words)) # заносим в список мн-во из тем и их "степень близости" c текстом
    return max(similarity, key=lambda x: x[1]) # обращаемся к кортежу и возвращаем тему с макс. "степенью близости" и ее "степень близости"


def comparing_texts():
    nlp = spacy.load('en_core_web_sm')
    database_texts_dir = 'C:/Users/Дина/Documents/универ/mag/database texts'
    new_file_path = input("Введите название нового файла с текстом: ")

    database_texts = {} # заводим словарь для базы: ключ - тема текста, значение - текст
    for file_name in os.listdir(database_texts_dir): # итерируемся по названиям файлов в папке с базированными текстами
        file_path = os.path.join(database_texts_dir, file_name) # полный путь к файлу (директория + имя файла)
        topic = file_name.split('.')[0] # вытаскиваем тему текста из названия файла (я решила сделать так, чтобы немного проще жилось...)
        text = read_file(file_path) # вызываем функцию чтения файла
        database_texts[topic] = text # ключ - тема, значение - текст

    new_text = read_file(new_file_path) # чиатем новый файл
    topic, similarity = count_similarity(database_texts, new_text, nlp) # сравниваем текст из базы с новым
    print(f'Текст больше всего похож на тему "{topic}"!')


comparing_texts()


KeyboardInterrupt



### Задача 2
Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

- составить список таких предлогов (РГ-80 вам в помощь)
- взять достаточно большой текст (можно большое художественное произведение)
- сделать морфоразбор этого текста
- Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.

Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести).

In [52]:
import pymorphy2

prepositions = ['по', 'с', 'со', 'в', 'во', 'за', 'меж', 'между', 'промеж', 'промежду', 'на', 'о', 'об', 'под', 'подо', 'согласно']



### Задача 3
Представим, что у вас есть файл с разборами conllu (можете взять любой, например, тут). Нужно просмотреть все примеры предложений с тегом dislocated и тегом discourse: напишите скрипт, который будет читать файл, находить все такие предложения и печатать: 1) сам текст предложения 2) слово, имеющее искомый тег. Если тег не был найден в файле, нужно об этом сообщить. Постарайтесь оформить вывод таким образом, чтобы это было удобно читать.

In [70]:
import pyconll


text = pyconll.load_from_file('social.conllu')
tags_found = 0

for sentence in text:
    for token in sentence:
        if token.deprel == 'dislocated' or token.deprel == 'discourse':
            tags_found += 1
            print(f"Предложение: {' '.join([token.form for token in sentence])}\nСлово с тегом {token.deprel}: {token.form}")

if tags_found == 0:
    print('Искомые теги не были найдены в файле.')

Предложение: Короче : столько - то " я как Я " , остальное " я как МЫ " , а если только первое , то это — привет психические расстройства .
Слово с тегом discourse: то
Предложение: - Ой-ой-ой , а сам - то с мамой спишь каждый день , как маленький !
Слово с тегом discourse: Ой-ой-ой
Предложение: - Ой-ой-ой , а сам - то с мамой спишь каждый день , как маленький !
Слово с тегом discourse: то
Предложение: - Ну ты подожди ещё , у неё и дети будут .
Слово с тегом discourse: Ну
Предложение: И они на ковре вертолете сваливают сначала на Ямайку , а потом в Бразилию , ну это там где много диких обезьян в белых штанах .
Слово с тегом discourse: ну


### Задача 4
Возьмите любой достаточно длинный (лучше новостной) текст. Любым известным инструментом извлеките именованные сущности из этого текста и выведите их списком по категориям (т.е. персоны вместе, локации вместе, организации вместе).

In [ ]:
import spacy

organizations, locations, people = [], [], []

with open('news.txt', 'r', encoding='utf-8') as f:
    text = f.read()

nlp = spacy.load('ru_core_news_sm')
doc = nlp(text)
lemmatized_tokens = [token.lemma_ for token in doc]
for ent in doc.ents:
    if ent.label_ == 'ORG':
        organizations.append(ent.text)
    elif ent.label_ == 'LOC':
        locations.append(ent.text)
    elif ent.label_ == 'PER':
        people.append(ent.text)

print(f'Вот найденные именованные сущности:\n{set(organizations)}')

In [82]:
import spacy

named_entity_categories = {}

with open('news.txt', 'r', encoding='utf-8') as f:
    text = f.read()

nlp = spacy.load('ru_core_news_sm')
doc = nlp(text)
for ent in doc.ents:
    if ent.label_ not in named_entity_categories:
        named_entity_categories[ent.label_] = set()
    named_entity_categories[ent.label_].add(ent.lemma_)

if named_entity_categories:
    for category, entity in named_entity_categories.items():
        print(f'{category}: {", ".join(entity)}')
else:
    print('Именованные сущности не были найдены.')

Именованные сущности не были найдены.
